In [61]:
import torch 
import torch.nn as nn
import fasttext as ft
import math
import torch.nn.functional as F

In [3]:
d_model = 10
new_dim = 5

In [62]:
class feedforward_Encoder(nn.Module):
    
    def ___init__(self):
        super(feedforward_Encoder,self).__init__()
        self.l1 = nn.Linear(10,40)
        self.l2 = nn.Linear(40,10)
        
    def forward(self,x):
        y = F.relu(self.l1(x))
        y = self.l2(y)
        return y
        
    

In [66]:
class Encoder():
    def __init__(self, vectorRepresentations):
        self.vectorRepresentations = vectorRepresentations
        self.d_model = d_model
        self.new_dim = new_dim
        self.positional_encodings = None
        self.keys = None
        self.values = None


    def PositionalEncoding(self,wordVecs):
        for pos in range(wordVecs.shape[0]):
            for i in range(wordVecs[pos].shape[0]):
                if i%2 == 0:
                    wordVecs[pos][i] = wordVecs[pos][i] + math.sin(pos/(10000**(2*i/self.d_model)))
                else:
                    wordVecs[pos][i] = wordVecs[pos][i] + math.cos(pos/(10000**(2*i/self.d_model))) 
                    
        self.positional_encodings = wordVecs
        return wordVecs


    def get_qkv_weights(self,r,c):
        query_weights = torch.rand((r,c))
        key_weights = torch.rand((r,c))
        value_weights = torch.rand((r,c))
        self.keys = key_weights
        self.values = value_weights
        
        return query_weights, key_weights, value_weights
    
    
    def get_keys_and_values(self):
        return self.keys, self.values
    
    
    
    def qkvs(self,vectorMatrix, new_dim):
        query_weights, key_weights, value_weights = self.get_qkv_weights(self.d_model,new_dim)
        return torch.matmul(vectorMatrix, query_weights), torch.matmul(vectorMatrix, key_weights), \
        torch.matmul(vectorMatrix, value_weights) 
        # Check for transposeness in matrix multiplication
    
    
    def qk_dotproducts(self,queries, keys):
        dotproduct_matrix = torch.Tensor([])
        for i in queries:
            dotproduct_vector = torch.Tensor([])
            for j in keys:
                dotproduct_vector = torch.cat([dotproduct_vector, torch.dot(i,j).reshape(-1)])
            dotproduct_matrix = torch.cat([dotproduct_matrix, dotproduct_vector.reshape(1,-1)])
        return dotproduct_matrix
    
    
    def getSoftmaxed_qkdp(self,qk_dotproductmatrix):
        sm = nn.Softmax(dim = 0)
        sm_matrix = torch.tensor([])
        for i in qk_dotproductmatrix:
            sm_matrix = torch.cat([sm_matrix, sm(i).reshape(1,-1)])
        return sm_matrix
    
    
    def getSoftmaxWeightedValues(self,softmaxed_qkdp, values):
        dim2_mat = torch.tensor([])
        dim3_mat = torch.tensor([])
        outer_loop_range = softmaxed_qkdp.shape[0]
        inner_loop_range = values.shape[0]
        for i in range(outer_loop_range):
            for j in range(inner_loop_range):
                dim2_mat = torch.cat([dim2_mat, (softmaxed_qkdp[i][j]*values[j]).reshape(-1)])
            dim3_mat = torch.cat([dim3_mat, dim2_mat.reshape(1,values.shape[0],values.shape[1])])
            dim2_mat = torch.tensor([]) 
        return dim3_mat
    
    
    
    def getWeightedSum(self,softmax_weighted_values):
        next_layer_input = torch.tensor([])
        for i in softmax_weighted_values:
            transposed_i = i.t()
            new_word_representation = torch.tensor([])
            for j in transposed_i:
                rowsum = j.sum()
                new_word_representation = torch.cat([new_word_representation, rowsum.reshape(-1)])
            next_layer_input = \
            torch.cat([next_layer_input, new_word_representation.reshape(1,new_word_representation.shape[0])])    
        return next_layer_input
        
    
    
    def returnRepresentation(self):
        pos_encoded = self.PositionalEncoding(self.vectorRepresentations)
        new_dim = self.new_dim
        queries, keys, values = self.qkvs(pos_encoded, new_dim)
        qk_dotproductmatrix = self.qk_dotproducts(queries, keys)
        d_k = keys.shape[1] # to be changed later to square root of 'key' vector dimension
        qk_dotproductmatrix/=d_k
        softmaxed_qkdp = self.getSoftmaxed_qkdp(qk_dotproductmatrix)
        softmax_weighted_values = self.getSoftmaxWeightedValues(softmaxed_qkdp, values)
        weightedSum = self.getWeightedSum(softmax_weighted_values)
        return weightedSum  
    
    
    def getW0(self):
        self.t = torch.randn(self.d_model, self.d_model).float()
        return self.t
    
    
    
    def multiHeadAttention(self, wordVecs, heads=2):
        listOfHeads = []
        op = torch.tensor([])
        for i in range(heads):
            temp = self.returnRepresentation()
            listOfHeads.append(temp)
    
        outputRepresentation = torch.tensor([])
        for i in range(listOfHeads[0].shape[0]):
            outputRepresentation = torch.cat([listOfHeads[0][i],listOfHeads[1][i]])
            op = torch.cat([op, outputRepresentation.reshape(1,outputRepresentation.shape[0])])
        
        W0 = self.getW0()
        projected_attention_vecs = torch.matmul(op, W0) 
        #Layer Normalisation
        layer_norm = nn.LayerNorm(projected_attention_vecs.size()[1])
        add_and_norm = layer_norm(projected_attention_vecs+self.positional_encodings)
    
        return add_and_norm
    
    
    def ff(self):
        received_representations = self.multiHeadAttention(self.vectorRepresentations)
        ffobj = feedforward_Encoder()
        activations = torch.tensor([])
        for i in received_representations:
            activations = torch.cat([activations, ffobj(i)])
            
        return activations
        
        
    
    
    def forward(self):
        return self.ff()


In [37]:
def getWordVectors(sentence):
    sentence = sentence.split(' ')
    vecs = torch.rand((len(sentence),10))
    return vecs

In [64]:
wordVecs = getWordVectors('Hi there this is nuts')
# wordVecs

In [39]:
for i in range(6):
    a = Encoder(wordVecs)
    wordVecs = a.forward()
#     print(wordVecs)

In [67]:
a = Encoder(wordVecs)
wordVecs = a.forward()

AttributeError: 'feedforward_Encoder' object has no attribute 'l1'

In [54]:
wordVecs

<bound method Encoder.ff of <__main__.Encoder object at 0x7f085d170710>>